In [2]:
from neo4j import GraphDatabase

In [4]:
uri = "neo4j+s://ed7c1538.databases.neo4j.io"
username = "neo4j"
password = "p0oXR2cgnZXajwX2Nm-Wk6rJXoNfv1QfLlFZAjQ9LUM"
auth=(username, password)
driver = GraphDatabase.driver(uri, auth=(username, password))

In [11]:
node_id = "Disease::DOID:184"

In [12]:
def query1(driver, node_id):
    query = """
        MATCH (n {id: $node_id})
        OPTIONAL MATCH (n)-[r1]->(target)
        WHERE r1.metaedge IN ['DlA', 'DaG']
        OPTIONAL MATCH (source)-[r2]->(n)
        WHERE r2.metaedge IN ['CpD', 'CtD']
        RETURN n, COLLECT(DISTINCT target) AS target_nodes, COLLECT(DISTINCT source) AS source_nodes
    """
    with driver.session() as session:
        result = session.run(query, node_id=node_id)
    
    return result

In [13]:
result = query1(driver, node_id)

In [14]:
def query2(driver):
    query = """
    MATCH (c:Compound)
    WHERE NOT (c)-[:CtD|:CpD]->(:Disease)
    
    OPTIONAL MATCH (c)-[cu:CuG]->(g:Gene)<-[ad:AdG]-(a:Anatomy)
    OPTIONAL MATCH (c)-[cd:CdG]->(g2:Gene)<-[au:AuG]-(a2:Anatomy)
    
    WHERE (cu IS NOT NULL AND ad IS NOT NULL) 
       OR (cd IS NOT NULL AND au IS NOT NULL)

    RETURN DISTINCT c.id AS compound_id, c.name AS compound_name,
           COLLECT(DISTINCT a.name) + COLLECT(DISTINCT a2.name) AS related_anatomies
    """

    # Execute the query and fetch results
    with driver.session() as session:
        results = session.run(query)
        compounds = [
            {"id": record["compound_id"], "name": record["compound_name"], "anatomies": record["related_anatomies"]}
            for record in results
        ]

    return compounds

In [15]:
compounds = query2(driver)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: CuG)} {position: line: 5, column: 28, offset: 98} for query: '\n    MATCH (c:Compound)\n    WHERE NOT (c)-[:CtD|:CpD]->(:Disease)\n    \n    OPTIONAL MATCH (c)-[cu:CuG]->(g:Gene)<-[ad:AdG]-(a:Anatomy)\n    OPTIONAL MATCH (c)-[cd:CdG]->(g2:Gene)<-[au:AuG]-(a2:Anatomy)\n    \n    WHERE (cu IS NOT NULL AND ad IS NOT NULL) \n       OR (cd IS NOT NULL AND au IS NOT NULL)\n\n    RETURN DISTINCT c.id AS compound_id, c.name AS compound_name,\n           COLLECT(DISTINCT a.name) + COLLECT(DISTINCT a2.name) AS related_anatomies\n    '
Received 